In [8]:
import pandas as pd

# Load the CSV file
file_path = 'Police_Department_Incident_Reports__Historical_2003_to_May_2018_20240514.csv'
data = pd.read_csv(file_path)

# Display the head of the DataFrame
print(data.head())


             PdId  IncidntNum  Incident Code       Category  \
0   4133422003074    41334220           3074        ROBBERY   
1   5118535807021    51185358           7021  VEHICLE THEFT   
2   4018830907021    40188309           7021  VEHICLE THEFT   
3  11014543126030   110145431          26030          ARSON   
4  10108108004134   101081080           4134        ASSAULT   

                Descript DayOfWeek        Date   Time PdDistrict Resolution  \
0  ROBBERY, BODILY FORCE    Monday  11/22/2004  17:50  INGLESIDE       NONE   
1      STOLEN AUTOMOBILE   Tuesday  10/18/2005  20:00       PARK       NONE   
2      STOLEN AUTOMOBILE    Sunday  02/15/2004  02:00   SOUTHERN       NONE   
3                  ARSON    Friday  02/18/2011  05:27  INGLESIDE       NONE   
4                BATTERY    Sunday  11/21/2010  17:00   SOUTHERN       NONE   

   ... Fix It Zones as of 2017-11-06  2 2  DELETE - HSOC Zones 2 2  \
0  ...                                NaN                      NaN   
1  ...

In [9]:
# Create a new DataFrame with the specified columns
new_table = data[['Category', 'X', 'Y', 'Date']]

# Sort the new DataFrame by date
new_table = new_table.sort_values(by='Date', ascending=False)
# Display the head of the sorted new DataFrame
print(new_table.head())


               Category           X          Y        Date
1371558    NON-CRIMINAL -122.503471  37.771688  12/31/2017
1365489         ROBBERY -122.417339  37.764366  12/31/2017
1346400  OTHER OFFENSES -122.419976  37.759368  12/31/2017
1363256   LARCENY/THEFT -122.417855  37.801970  12/31/2017
1248649  SUSPICIOUS OCC -122.403405  37.775421  12/31/2017


In [10]:
!pip install folium

In [11]:
# Filter the data to include only records with Date after 2015
new_table['Date'] = pd.to_datetime(new_table['Date'])
filtered_table = new_table[new_table['Date'] > '2015-01-01']

# Display the head of the filtered DataFrame
print(filtered_table.head())


               Category           X          Y       Date
1371558    NON-CRIMINAL -122.503471  37.771688 2017-12-31
1365489         ROBBERY -122.417339  37.764366 2017-12-31
1346400  OTHER OFFENSES -122.419976  37.759368 2017-12-31
1363256   LARCENY/THEFT -122.417855  37.801970 2017-12-31
1248649  SUSPICIOUS OCC -122.403405  37.775421 2017-12-31


In [12]:
import folium
from folium.plugins import MarkerCluster

# Create a base map
base_map = folium.Map(location=[37.77, -122.42], zoom_start=12)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(base_map)

# Add a marker for every record in the filtered data, use a clustered view
for index, row in filtered_table.iterrows():
    folium.Marker(
        location=[row["Y"], row["X"]],
    ).add_to(marker_cluster)

# Save the map to an HTML file
base_map.save('heatmap.html')

# Display the map in the notebook (if running in a Jupyter environment)
base_map

KeyboardInterrupt: 

In [ ]:
!pip install shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.2 MB/s eta 0:00:00a 0:00:010m


In [31]:
import pandas as pd

# Load the CSV file
file_path = "/Users/hudzah/Documents/Coding/sf-safe/Police_Department_Incident_Reports__2018_to_Present_20240514.csv"
data = pd.read_csv(file_path)

# Display the head of the DataFrame
print(data.head())

        Incident Datetime Incident Date Incident Time  Incident Year  \
0  2023/03/13 11:41:00 PM    2023/03/13         23:41           2023   
1  2023/03/01 05:02:00 AM    2023/03/01         05:02           2023   
2  2023/03/13 01:16:00 PM    2023/03/13         13:16           2023   
3  2023/03/13 10:59:00 AM    2023/03/13         10:59           2023   
4  2023/03/14 06:44:00 PM    2023/03/14         18:44           2023   

  Incident Day of Week         Report Datetime        Row ID  Incident ID  \
0               Monday  2023/03/13 11:41:00 PM  125373607041      1253736   
1            Wednesday  2023/03/11 03:40:00 PM  125379506374      1253795   
2               Monday  2023/03/13 01:17:00 PM  125357107041      1253571   
3               Monday  2023/03/13 11:00:00 AM  125355107041      1253551   
4              Tuesday  2023/03/14 06:45:00 PM  125402407041      1254024   

   Incident Number  CAD Number  ... Longitude Point Neighborhoods  \
0        230167874         NaN  ...

In [41]:
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 27.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 28.0 MB/s eta 0:00:0000:0100:01


In [45]:
import csv
import json
from shapely import wkt

csv.field_size_limit(1000000)

feature_collection = {"type": "FeatureCollection", "features": []}

# Open the CSV file and read the data
with open("Analysis_Neighborhoods_20240514.csv", "r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        # Extract the WKT geometry and neighborhood from the CSV row
        geometry_wkt = row["the_geom"]
        neighborhood = row["nhood"]

        geometry = wkt.loads(geometry_wkt)

        geojson_geometry = json.loads(json.dumps(geometry.__geo_interface__))

        feature = {
            "type": "Feature",
            "geometry": geojson_geometry,
            "properties": {"neighborhood": neighborhood},
        }

        feature_collection["features"].append(feature)

# Save the GeoJSON data to a file
with open("output.geojson", "w") as file:
    json.dump(feature_collection, file)

In [47]:
import pandas as pd
import folium

# Load the CSV file
file_path = "Police_Department_Incident_Reports__2018_to_Present_20240514.csv"
data = pd.read_csv(file_path)

# calculating total number of incidents per neighborhood
crime_data_neighborhood = pd.DataFrame(data["Neighborhoods"].value_counts().astype(float))
crime_data_neighborhood.to_json("crimeagg_neighborhood.json")
crime_data_neighborhood = crime_data_neighborhood.reset_index()
crime_data_neighborhood.columns = ["Neighborhood", "Number"]

# definition of the boundaries in the map
district_geo = r"sfpddistricts.geojson"

# creation of the choropleth map
SF_COORDINATES = [37.77, -122.42]
map1 = folium.Map(location=SF_COORDINATES, zoom_start=12)
folium.Choropleth(
    geo_data=district_geo,
    name="choropleth",
    data=crime_data_neighborhood,
    columns=["Neighborhood", "Number"],
    key_on="feature.properties.neighborhood",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Number of incidents per neighborhood",
).add_to(map1)

# Save the map to an HTML file
map1.save('neighborhood_choropleth.html')

# Display the map in the notebook (if running in a Jupyter environment)
map1

ValueError: key_on `'properties.Neighborhood'` not found in GeoJSON.